# model

## 微调+Morris

### 修改

In [27]:
import pandas as pd, numpy as np, os, re, psutil, shap, matplotlib.pyplot as plt
from xgboost import XGBRegressor
from interpret.blackbox import MorrisSensitivity
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score,
    mean_squared_error, mean_absolute_error, median_absolute_error
)
import matplotlib

# —— 字体配置：宋体 + Times ——
config = {
    "font.family": 'serif',
    "font.serif": ['SimSun'],
    "mathtext.fontset": 'stix',
    "font.size": 12,
    "axes.unicode_minus": False
}
matplotlib.rcParams.update(config)

# —— 路径配置 ——
INPUT = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\data\var_attri_data_interp_cleaned.csv"
OUTPUT_DIR = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def sanitize(s):
    return re.sub(r'[<>:"/\\|?*]', '_', s)

def check_memory(gb=1):
    avail = psutil.virtual_memory().available / (1024**3)
    if avail < gb:
        raise MemoryError(f"内存不足: {avail:.2f} GB")
        
def analyze(var, df, output_dir, fine_tune_regions=["CHN","R10CHINA+"]):
    id_cols = ['Model','Scenario','Region','Year', var]
    # 新增：同时剔除所有包含自变量名的列
    vars_cols = [
        c for c in df.columns
        if c not in id_cols and var not in c
    ]
    data = df[id_cols + vars_cols].dropna()
    X, y = data[vars_cols], data[var].values

    # —— 模型：XGBoost ——
    et = XGBRegressor(
        n_estimators=40,
        max_depth=6,
        min_child_weight=5,
        colsample_bytree=0.7,
        random_state=42,
        tree_method='auto',
        verbosity=0
    )
    et.fit(X, y)
    mask = data['Region'].isin(fine_tune_regions)
    X_sub, y_sub = X[mask], y[mask]
    et.set_params(n_estimators=60)
    et.fit(X_sub, y_sub)

    check_memory()

    # ——— 性能评估
    Xtr, Xte, ytr, yte = train_test_split(X_sub, y_sub, test_size=0.3, random_state=42)
    preds_te = et.predict(Xte)
    metrics = {
        "R2_in": r2_score(ytr, et.predict(Xtr)),
        "R2_out": r2_score(yte, preds_te),
        "EVS_out": explained_variance_score(yte, preds_te),
        "MSE_out": mean_squared_error(yte, preds_te),
        "MAE_out": mean_absolute_error(yte, preds_te),
        "MedAE_out": median_absolute_error(yte, preds_te),
    }
    tag = sanitize(var)
    outdir = os.path.join(output_dir, tag)
    os.makedirs(outdir, exist_ok=True)
    pd.DataFrame([metrics]).to_csv(
        os.path.join(outdir, f"{tag}_模型指标.csv"),
        index=False, encoding='utf-8-sig'
    )

    # ——— 特征重要性
    imp = et.feature_importances_
    imp_df = pd.DataFrame({
        "变量": vars_cols,
        "相对重要性(%)": np.round(imp / imp.sum() * 100, 4)
    }).nlargest(10, "相对重要性(%)")
    imp_df.to_csv(
        os.path.join(outdir, f"{tag}_变量相对重要性.csv"),
        index=False, encoding='utf-8-sig'
    )

    # ——— SHAP 分析：注意采样用全部训练特征
    sample = X_sub.sample(n=min(100, len(X_sub)), random_state=42)
    expl = shap.TreeExplainer(et, feature_perturbation="auto")
    shap_vals = expl.shap_values(sample, approximate=True, check_additivity=False)

    # 只画 top_feats
    top_feats = imp_df["变量"].tolist()
    top_idx = [sample.columns.get_loc(f) for f in top_feats]

    # SHAP summary_plot 只传 top_feats 的值和名字
    shap.summary_plot(
        shap_vals[:, top_idx],
        sample[top_feats],
        feature_names=top_feats,
        show=False, max_display=10
    )
    fig = plt.gcf()
    ax = fig.axes[0]; ax.set_xlabel(""); ax.set_ylabel("")
    cbar = fig.axes[-1]; cbar.set_ylabel(""); cbar.set_yticklabels([]); cbar.set_xticklabels([])

    # 只对top_feats计算均值/标准差
    mean_shap = np.abs(shap_vals[:, top_idx]).mean(axis=0)
    ordered_idx = np.argsort(mean_shap)[::-1]
    for y_loc, idx in enumerate(ordered_idx):
        mv = mean_shap[idx]
        ax.hlines(y=y_loc, xmin=mv - 0.01, xmax=mv + 0.01,
                  color="black", linewidth=12, zorder=20)

    plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"{tag}_SHAP.png"), dpi=300)
    plt.close()
    pd.DataFrame({
        "变量": [top_feats[i] for i in ordered_idx],
        "SHAP均值": np.round(mean_shap[ordered_idx], 6)
    }).to_csv(
        os.path.join(outdir, f"{tag}_mean_SHAP.csv"),
        index=False, encoding='utf-8-sig'
    )

    mean_shap_df = pd.DataFrame({
        "变量": [top_feats[i] for i in ordered_idx],
        "均值_SHAP": np.round(mean_shap[ordered_idx], 6),
        "特征重要性": np.round(imp_df["相对重要性(%)"].values[ordered_idx] / 100, 6),
        "SHAP_标准差": np.round(np.std(shap_vals[:, top_idx][:, ordered_idx], axis=0), 6)
    })
    mean_shap_df.to_csv(os.path.join(outdir, f"{tag}_SHAP_vs_Imp.csv"),
                        index=False, encoding='utf-8-sig')

    # 📊 图1. SHAP vs 特征重要性柱状图
    fig, ax = plt.subplots(figsize=(8, 6))
    df_plot = mean_shap_df.sort_values("均值_SHAP", ascending=True)
    y = np.arange(len(df_plot))
    ax.barh(y - 0.2, df_plot["均值_SHAP"], height=0.4, label="SHAP均值")
    ax.barh(y + 0.2, df_plot["特征重要性"], height=0.4, label="特征重要性")
    ax.set_yticks(y)
    ax.set_yticklabels(df_plot["变量"])
    ax.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"{tag}_SHAP_vs_Imp_bar.png"), dpi=300)
    plt.close()

    # 📈 图2. Morris µ* vs σ 散点图
    fig, ax = plt.subplots(figsize=(6, 6))
    sc = ax.scatter(
        mean_shap_df["均值_SHAP"],
        mean_shap_df["SHAP_标准差"],
        s=100 * mean_shap_df["特征重要性"],
        c=mean_shap_df["特征重要性"],
        cmap="viridis",
        alpha=0.8,
        edgecolors='k'
    )
    for _, row in mean_shap_df.iterrows():
        ax.text(row["均值_SHAP"], row["SHAP_标准差"], row["变量"], fontsize=8)
    ax.set_xlabel("Morris µ* (SHAP均值)")
    ax.set_ylabel("σ (SHAP标准差)")
    plt.colorbar(sc, label="特征重要性")
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"{tag}_Morris_scatter.png"), dpi=300)
    plt.close()

    # 📉 图3. SHAP Dependence Plot（前 3 个变量）
    for i, feat in enumerate([top_feats[i] for i in ordered_idx[:3]]):
        interaction_index = 'auto'
        if interaction_index == 'auto':
            interactions = shap.utils.approximate_interactions(
                feat, shap_vals, sample
            )
            interaction_feat = sample.columns[interactions[0]]
        else:
            interaction_feat = interaction_index

        x = sample[feat].values
        y = shap_vals[:, sample.columns.get_loc(feat)]
        color = sample[interaction_feat].values if interaction_feat != feat else None

        shap.dependence_plot(
            sample.columns.get_loc(feat), shap_vals, sample,
            feature_names=sample.columns.tolist(),
            interaction_index=interaction_index, show=False
        )
        plt.tight_layout()
        png_path = os.path.join(outdir, f"{tag}_SHAP_dependence_{sanitize(feat)}.png")
        plt.savefig(png_path, dpi=300)
        plt.close()
        
        data = {
            feat: x,
            f"SHAP_{feat}": y,
        }
        if color is not None:
            data[interaction_feat] = color
        df = pd.DataFrame(data)
        csv_path = os.path.join(outdir, f"{tag}_SHAP_dependence_{sanitize(feat)}.csv")
        df.to_csv(csv_path, index=False)

    print(f"[✔] {var} 使用 XGBoost 分析完成，结果保存在：{outdir}")


## Sobol 漏斗式验证

In [28]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor  # ✅ 替换为XGBoost

def run_sobol_analysis(X, y, var_names, output_dir, tag="fei"):
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X[var_names])

    problem = {
        'num_vars': len(var_names),
        'names': var_names,
        'bounds': [[0, 1]] * len(var_names)
    }

    # 采样 + 模拟
    param_values = saltelli.sample(problem, 512, calc_second_order=True)
    # ==== 替换为XGBRegressor ====
    model = XGBRegressor(n_estimators=100, max_depth=6, random_state=42, verbosity=0)
    model.fit(X_scaled, y)
    Y = model.predict(param_values)

    # Sobol 分析
    sobol_result = sobol.analyze(problem, Y, calc_second_order=True, print_to_console=False)

    # 保存结果
    df_out = pd.DataFrame({
        "变量": var_names,
        "一阶S1": np.round(sobol_result['S1'], 4),
        "总效ST": np.round(sobol_result['ST'], 4),
        "S1_conf": np.round(sobol_result['S1_conf'], 4),
        "ST_conf": np.round(sobol_result['ST_conf'], 4)
    })

    outpath = os.path.join(output_dir, f"{tag}_sobol.csv")
    df_out.to_csv(outpath, index=False, encoding='utf-8-sig')
    print(f"[✔] Sobol 总效应分析完成，结果已保存至：{outpath}")

In [29]:
def subgroup_shap_analysis_model(var, df, output_dir, fine_tune_regions):
    df_china = df[df['Region'].isin(fine_tune_regions)].copy()
    model_vals = df_china["Model"].unique()

    for model_name in model_vals:
        df_sub = df_china[df_china["Model"] == model_name].copy()
        if len(df_sub) < 40:
            print(f"[跳过] Model={model_name} 样本过少（{len(df_sub)} 条）")
            continue

        try:
            print(f"[▶] 正在分析 Model={model_name}...")
            sub_output = os.path.join(output_dir, f"China_Model_{sanitize(str(model_name))}")
            # analyze函数需兼容XGBoost，参数顺序保持一致
            analyze(var, df_sub, sub_output, fine_tune_regions)
        except Exception as e:
            print(f"[❌] Model={model_name} 分析失败：{e}")


# fei

In [49]:
# 加载数据并格式转换
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# 单变量调用
analyze('fei', df, OUTPUT_DIR)

[✔] fei 使用 XGBoost 分析完成，结果保存在：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei


## 子集特征再训练与验证

In [30]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(var, df, feature_subset, subset_name, output_dir):
    id_cols = ['Model','Scenario','Region','Year', var]
    df = df[id_cols + feature_subset].dropna()
    X_all, y_all = df[feature_subset], df[var].values

    # 1️⃣ 全样本训练（基线模型40棵树）
    xgb_reg_base = xgb.XGBRegressor(
        n_estimators=40,
        max_depth=6,
        min_child_weight=5,
        colsample_bytree=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg_base.fit(X_all, y_all)

    # 2️⃣ 中国样本追加微调（再加20树）
    mask_china = df['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg_ft = xgb.XGBRegressor(
            n_estimators=60,  # 总共60棵树，前40为基线
            max_depth=6,
            min_child_weight=5,
            colsample_bytree=0.8,
            random_state=42,
            verbosity=0,
            n_jobs=-1,
            tree_method='auto'
        )
        xgb_reg_ft.fit(X_china, y_china, xgb_model=xgb_reg_base.get_booster())
        final_model = xgb_reg_ft
    else:
        final_model = xgb_reg_base

    # 3️⃣ 训练/测试集评估（在全部样本上）
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, final_model.predict(Xtr)),
        "R2_out": r2_score(yte, final_model.predict(Xte)),
        "EVS_out": explained_variance_score(yte, final_model.predict(Xte)),
        "MSE_out": mean_squared_error(yte, final_model.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, final_model.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, final_model.predict(Xte)),
    }
    return metrics

# =================== 主体流程 ===================
linear_feats    = ["fel", "pec", "fee"]
nonlinear_feats = ["seeh", "pe", "eced"]
shap_top5_feats = ["seeh", "pe", "eced", "fel", "pec", "fee", "peo", "seeg", "emcoen", "eces"]

df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

target_var = 'fei'
outdir = os.path.join(OUTPUT_DIR, "fei")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(evaluate_subset(target_var, df, linear_feats,    "线性主效变量",    outdir))
results.append(evaluate_subset(target_var, df, nonlinear_feats, "非线性交互变量", outdir))
results.append(evaluate_subset(target_var, df, shap_top5_feats, "SHAP前五变量",   outdir))

summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(outdir, "fei_子集建模比较结果.csv"), index=False, encoding='utf-8-sig')
print("✅ 子集建模评估完成（追加增树微调），结果已保存至：", outdir)

✅ 子集建模评估完成（追加增树微调），结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei


## Sobol 漏斗式验证

In [28]:
# 选定目标变量和特征组合（例如基于 Morris 筛选）
target_var = "fei"
sobol_vars = ["seeh", "pe", "eced"]  # 可根据 Morris σ 高变量选择

# 数据准备
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# 仅取中国区域数据
fine_tune_regions = ["CHN", "R10CHINA+"]
df_china = df[df['Region'].isin(fine_tune_regions)].dropna(subset=[target_var] + sobol_vars)

X = df_china[sobol_vars]
y = df_china[target_var].values

In [29]:
run_sobol_analysis(X, y, sobol_vars, OUTPUT_DIR, tag="fei_sobol")

C:\Users\phc\AppData\Local\Temp\ipykernel_1800\633809096.py:17: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 512, calc_second_order=True)


[✔] Sobol 总效应分析完成，结果已保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei_sobol_sobol.csv


C:\Users\phc\AppData\Roaming\Python\Python310\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


## 模型降维&交互建模

### 只用eced

In [9]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(var, df, feature_subset, subset_name, output_dir, max_depth=4):
    id_cols = ['Model', 'Scenario', 'Region', 'Year', var]
    df = df[id_cols + feature_subset].dropna()
    X_all, y_all = df[feature_subset], df[var].values

    # Step 1️⃣ 全部样本训练
    xgb_reg = xgb.XGBRegressor(
        n_estimators=40,
        max_depth=max_depth,
        min_child_weight=5,
        colsample_bytree=0.8,
        subsample=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # Step 2️⃣ 如果中国子样本充足，追加树微调
    mask_china = df['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 追加20棵树
        xgb_reg.fit(X_china, y_china, xgb_model=xgb_reg.get_booster())

    # Step 3️⃣ 划分训练/测试集评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out": r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out": explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out": mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }
    return metrics

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 只用eced做回归 ===
feature_subset = ['eced']
target_var = 'fei'
outdir = os.path.join(OUTPUT_DIR, "fei")
os.makedirs(outdir, exist_ok=True)

results = []
# 你可以for循环max_depth=4,5,6多次，这里以4为例
results.append(evaluate_subset(target_var, df, feature_subset, "只用eced", outdir, max_depth=4))

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(outdir, "fei_eced单变量回归结果.csv"), index=False, encoding='utf-8-sig')
print("✅ 只用eced变量回归评估完成，结果已保存至：", outdir)

✅ 只用eced变量回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei


### eced进行三段哑变量回归+eced

In [10]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(var, df, feature_subset, subset_name, output_dir, max_depth=4):
    id_cols = ['Model', 'Scenario', 'Region', 'Year', var]
    df = df[id_cols + feature_subset].dropna()
    X_all, y_all = df[feature_subset], df[var].values

    # Step 1️⃣ 全部样本训练
    xgb_reg = xgb.XGBRegressor(
        n_estimators=40,
        max_depth=max_depth,
        min_child_weight=5,
        colsample_bytree=0.8,
        subsample=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # Step 2️⃣ “追加树”微调到中国子样本（而不是覆盖！）
    mask_china = df['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 追加20棵树
        xgb_reg.fit(X_china, y_china, xgb_model=xgb_reg.get_booster())

    # Step 3️⃣ 拆分测试集评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out": r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out": explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out": mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }
    return metrics

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 构造三段哑变量 ===
df['ec_low'] = (df['eced'] <= 6000).astype(int)
df['ec_mid'] = ((df['eced'] > 6000) & (df['eced'] <= 9000)).astype(int)
df['ec_high'] = (df['eced'] > 9000).astype(int)

# === 三段哑变量 + 原始eced做回归 ===
feature_subset = ['ec_low', 'ec_mid', 'ec_high', 'eced']
target_var = 'fei'
outdir = os.path.join(OUTPUT_DIR, "fei")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(
    evaluate_subset(target_var, df, feature_subset, "eced三段哑变量+原始", outdir, max_depth=4)
)

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(outdir, "fei_eced三段哑变量加原始回归结果.csv"), index=False, encoding='utf-8-sig')
print("✅ eced三段哑变量+原始变量回归评估完成，结果已保存至：", outdir)

✅ eced三段哑变量+原始变量回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei


### eced、pe、seeh

In [11]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(var, df, feature_subset, subset_name, output_dir, max_depth=4):
    id_cols = ['Model', 'Scenario', 'Region', 'Year', var]
    df = df[id_cols + feature_subset].dropna()
    X_all, y_all = df[feature_subset], df[var].values

    # Step 1️⃣ 全部样本训练（n_estimators=40）
    xgb_reg = xgb.XGBRegressor(
        n_estimators=40,
        max_depth=max_depth,
        min_child_weight=5,
        colsample_bytree=0.8,
        subsample=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # Step 2️⃣ 追加树微调到中国子样本（追加20棵树）
    mask_china = df['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 追加20棵树
        xgb_reg.fit(X_china, y_china, xgb_model=xgb_reg.get_booster())

    # Step 3️⃣ 拆分测试集评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out": r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out": explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out": mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }
    return metrics

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 只用eced、pe、seeh做回归 ===
feature_subset = ['eced', 'pe', 'seeh']
target_var = 'fei'
outdir = os.path.join(OUTPUT_DIR, "fei")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(evaluate_subset(target_var, df, feature_subset, "eced+pe+seeh", outdir, max_depth=4))

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(outdir, "fei_eced_pe_seeh回归结果.csv"), index=False, encoding='utf-8-sig')
print("✅ eced+pe+seeh变量回归评估完成，结果已保存至：", outdir)

✅ eced+pe+seeh变量回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei


### eced、pe、seeh+pe × fet

In [12]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(var, df, feature_subset, subset_name, output_dir, max_depth=4):
    id_cols = ['Model', 'Scenario', 'Region', 'Year', var]
    df = df[id_cols + feature_subset].dropna()
    X_all, y_all = df[feature_subset], df[var].values

    # Step 1️⃣ 全部样本训练（如40棵树）
    xgb_reg = xgb.XGBRegressor(
        n_estimators=40,
        max_depth=max_depth,
        min_child_weight=5,
        colsample_bytree=0.8,
        subsample=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # Step 2️⃣ “追加树”微调到中国子样本（如追加20棵树）
    mask_china = df['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 追加20棵树
        xgb_reg.fit(X_china, y_china, xgb_model=xgb_reg.get_booster())

    # Step 3️⃣ 拆分测试集评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out": r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out": explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out": mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }
    return metrics

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 构造交互项 ===
df['pe_fet'] = df['pe'] * df['fet']

# === 特征集 ===
feature_subset = ['eced', 'pe', 'seeh', 'pe_fet']
target_var = 'fei'
outdir = os.path.join(OUTPUT_DIR, "fei")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(evaluate_subset(target_var, df, feature_subset, "eced+pe+seeh+pe_fet", outdir, max_depth=4))

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(outdir, "fei_eced_pe_seeh_pefet回归结果.csv"), index=False, encoding='utf-8-sig')
print("✅ eced+pe+seeh+pe_fet变量回归评估完成，结果已保存至：", outdir)

✅ eced+pe+seeh+pe_fet变量回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei


### eced、pe、seeh+seeh×eced

In [13]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(var, df, feature_subset, subset_name, output_dir, max_depth=4):
    id_cols = ['Model', 'Scenario', 'Region', 'Year', var]
    df = df[id_cols + feature_subset].dropna()
    X_all, y_all = df[feature_subset], df[var].values

    # 1️⃣ 全样本训练
    xgb_reg = xgb.XGBRegressor(
        n_estimators=40,
        max_depth=max_depth,
        min_child_weight=5,
        colsample_bytree=0.8,
        subsample=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # 2️⃣ 追加树微调到中国子样本（不覆盖）
    mask_china = df['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20
        xgb_reg.fit(X_china, y_china, xgb_model=xgb_reg.get_booster())

    # 3️⃣ 拆分测试集评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out": r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out": explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out": mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }
    return metrics

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 构造交互项 ===
df['seeh_eced'] = df['seeh'] * df['eced']

# === 特征集 ===
feature_subset = ['eced', 'pe', 'seeh', 'seeh_eced']
target_var = 'fei'
outdir = os.path.join(OUTPUT_DIR, "fei")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(
    evaluate_subset(target_var, df, feature_subset, "eced+pe+seeh+seeh_eced", outdir, max_depth=4)
)

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(outdir, "fei_eced_pe_seeh_seeheced回归结果.csv"), index=False, encoding='utf-8-sig')
print("✅ eced+pe+seeh+seeh_eced变量回归评估完成，结果已保存至：", outdir)

✅ eced+pe+seeh+seeh_eced变量回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei


### eced、pe、seeh+pe × fet+seeh×eced

In [14]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(var, df, feature_subset, subset_name, output_dir, max_depth=4):
    id_cols = ['Model', 'Scenario', 'Region', 'Year', var]
    df = df[id_cols + feature_subset].dropna()
    X_all, y_all = df[feature_subset], df[var].values

    # Step 1️⃣ 全样本训练
    xgb_reg = xgb.XGBRegressor(
        n_estimators=40,
        max_depth=max_depth,
        min_child_weight=5,
        colsample_bytree=0.8,
        subsample=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # Step 2️⃣ 追加树微调到中国子样本（追加20棵树）
    mask_china = df['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 追加20棵树
        xgb_reg.fit(X_china, y_china, xgb_model=xgb_reg.get_booster())

    # Step 3️⃣ 拆分测试集评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out": r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out": explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out": mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }
    return metrics

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 构造交互项 ===
df['pe_fet'] = df['pe'] * df['fet']
df['seeh_eced'] = df['seeh'] * df['eced']

# === 特征集 ===
feature_subset = ['eced', 'pe', 'seeh', 'pe_fet', 'seeh_eced']
target_var = 'fei'
outdir = os.path.join(OUTPUT_DIR, "fei")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(evaluate_subset(target_var, df, feature_subset, "eced+pe+seeh+pe_fet+seeh_eced", outdir, max_depth=4))

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(outdir, "fei_eced_pe_seeh_pefet_seeheced回归结果.csv"), index=False, encoding='utf-8-sig')
print("✅ eced+pe+seeh+pe_fet+seeh_eced变量回归评估完成，结果已保存至：", outdir)

✅ eced+pe+seeh+pe_fet+seeh_eced变量回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei


# peo

In [51]:
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# 单变量调用
analyze('peo', df, OUTPUT_DIR)

[✔] peo 使用 XGBoost 分析完成，结果保存在：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\peo


## 子集特征再训练与验证

In [31]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)

def evaluate_subset(var, df, feature_subset, subset_name, output_dir):
    id_cols = ['Model','Scenario','Region','Year', var]
    df = df[id_cols + feature_subset].dropna()
    X_all, y_all = df[feature_subset], df[var].values

    # 1️⃣ 全样本基线模型（40棵树）
    xgb_reg_base = xgb.XGBRegressor(
        n_estimators=40,
        max_depth=6,
        min_child_weight=5,
        colsample_bytree=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg_base.fit(X_all, y_all)

    # 2️⃣ 中国子样本增树微调
    mask_china = df['Region'].isin(["CHN", "R10CHINA+"])
    final_model = xgb_reg_base
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg_finetune = xgb.XGBRegressor(
            n_estimators=60,  # 增加到60棵树
            max_depth=6,
            min_child_weight=5,
            colsample_bytree=0.8,
            random_state=42,
            verbosity=0,
            n_jobs=-1,
            tree_method='auto'
        )
        # 关键：用 xgb_model=基线模型booster，增树而非覆盖
        xgb_reg_finetune.fit(X_china, y_china, xgb_model=xgb_reg_base.get_booster())
        final_model = xgb_reg_finetune

    # 3️⃣ 全样本训练/测试集评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, final_model.predict(Xtr)),
        "R2_out": r2_score(yte, final_model.predict(Xte)),
        "EVS_out": explained_variance_score(yte, final_model.predict(Xte)),
        "MSE_out": mean_squared_error(yte, final_model.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, final_model.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, final_model.predict(Xte)),
    }
    return metrics

In [32]:
linear_feats = ["seeh", "seec","pecwc"]
nonlinear_feats = ["pe", "seeo","fel"]
shap_top5_feats = ["pe", "seeo", "fel", "fet", "seeh","seec","pecwc","seegwoc","pen","ecese"]

In [33]:
# 加载数据
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

target_var = 'peo'
outdir = os.path.join(OUTPUT_DIR, "peo")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(evaluate_subset(target_var, df, linear_feats, "线性主效变量", outdir))
results.append(evaluate_subset(target_var, df, nonlinear_feats, "非线性交互变量", outdir))
results.append(evaluate_subset(target_var, df, shap_top5_feats, "SHAP前十变量", outdir))

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(outdir, "peo_子集建模比较结果.csv"), index=False, encoding='utf-8-sig')
print("✅ 子集建模评估完成，结果已保存至：", outdir)

✅ 子集建模评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\peo


## Sobol 漏斗式验证

In [57]:
# 选定目标变量和特征组合（例如基于 Morris 筛选）
target_var = "peo"
sobol_vars = ["pe", "seeo","fel"]  # 可根据 Morris σ 高变量选择

# 数据准备
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# 仅取中国区域数据
fine_tune_regions = ["CHN", "R10CHINA+"]
df_china = df[df['Region'].isin(fine_tune_regions)].dropna(subset=[target_var] + sobol_vars)

X = df_china[sobol_vars]
y = df_china[target_var].values

In [58]:
run_sobol_analysis(X, y, sobol_vars, OUTPUT_DIR, tag="peo_sobol")

C:\Users\phc\AppData\Local\Temp\ipykernel_1800\633809096.py:17: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 512, calc_second_order=True)


[✔] Sobol 总效应分析完成，结果已保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\peo_sobol_sobol.csv


C:\Users\phc\AppData\Roaming\Python\Python310\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


## 模型降维&交互建模

### 只用fel

In [15]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(var, df, feature_subset, subset_name, output_dir):
    id_cols = ['Model','Scenario','Region','Year', var]
    df_subset = df[id_cols + feature_subset].dropna()
    X_all, y_all = df_subset[feature_subset], df_subset[var].values

    # 1️⃣ 全样本训练
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=5,
        min_child_weight=5,
        colsample_bytree=1.0,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # 2️⃣ 追加树微调到中国子样本
    mask_china = df_subset['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 比如再追加20棵树
        xgb_reg.fit(X_china, y_china, xgb_model=xgb_reg.get_booster())

    # 3️⃣ 拆分测试集评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out": r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out": explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out": mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }
    return metrics

# === 只用fel做回归 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

feature_subset = ["fel"]
target_var = 'peo'
outdir = os.path.join(OUTPUT_DIR, "peo")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(
    evaluate_subset(
        target_var,
        df,
        feature_subset,
        "fel单变量",
        outdir
    )
)

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(
    os.path.join(outdir, "peo_fel单变量回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ fel单变量回归评估完成，结果已保存至：", outdir)


✅ fel单变量回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\peo


### fel进行三段哑变量回归+fel

In [16]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(var, df, feature_subset, subset_name, output_dir):
    id_cols = ['Model','Scenario','Region','Year', var]
    df_subset = df[id_cols + feature_subset].dropna()
    X_all, y_all = df_subset[feature_subset], df_subset[var].values

    # 1️⃣ 全部样本训练
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=5,
        min_child_weight=5,
        colsample_bytree=1.0,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # 2️⃣ 追加树微调到中国子样本（而不是覆盖！）
    mask_china = df_subset['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 追加20棵树
        xgb_reg.fit(X_china, y_china, xgb_model=xgb_reg.get_booster())

    # 3️⃣ 拆分测试集评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out": r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out": explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out": mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }
    return metrics

# === 构造fel三段哑变量 + 原始fel ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

df['fel_low'] = (df['fel'] <= 15).astype(int)
df['fel_mid'] = ((df['fel'] > 15) & (df['fel'] <= 35)).astype(int)
df['fel_high'] = (df['fel'] > 35).astype(int)

feature_subset = ["fel_low", "fel_mid", "fel_high", "fel"]
target_var = 'peo'
outdir = os.path.join(OUTPUT_DIR, "peo")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(
    evaluate_subset(
        target_var,
        df,
        feature_subset,
        "fel三段哑变量+原始",
        outdir
    )
)

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(
    os.path.join(outdir, "peo_fel三段哑变量加原始回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ fel三段哑变量+原始变量回归评估完成，结果已保存至：", outdir)

✅ fel三段哑变量+原始变量回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\peo


### fel+ pe + seeo

In [17]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(var, df, feature_subset, subset_name, output_dir):
    id_cols = ['Model','Scenario','Region','Year', var]
    df_subset = df[id_cols + feature_subset].dropna()
    X_all, y_all = df_subset[feature_subset], df_subset[var].values

    # 1️⃣ 全部样本训练
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=5,
        min_child_weight=5,
        colsample_bytree=1.0,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # 2️⃣ 追加树微调到中国子样本（而不是覆盖！）
    mask_china = df_subset['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 追加20棵树
        xgb_reg.fit(X_china, y_china, xgb_model=xgb_reg.get_booster())

    # 3️⃣ 拆分测试集评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out": r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out": explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out": mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }
    return metrics

# === 只用 fel, pe, seeo 做回归 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

feature_subset = ["fel", "pe", "seeo"]
target_var = 'peo'
outdir = os.path.join(OUTPUT_DIR, "peo")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(
    evaluate_subset(
        target_var,
        df,
        feature_subset,
        "fel+pe+seeo",
        outdir
    )
)

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(
    os.path.join(outdir, "peo_fel_pe_seeo回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ fel+pe+seeo变量回归评估完成，结果已保存至：", outdir)

✅ fel+pe+seeo变量回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\peo


### fel+ pe + seeo+fel*pe

In [18]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(var, df, feature_subset, subset_name, output_dir):
    id_cols = ['Model','Scenario','Region','Year', var]
    df_subset = df[id_cols + feature_subset].dropna()
    X_all, y_all = df_subset[feature_subset], df_subset[var].values

    # 1️⃣ 全部样本训练
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=5,
        min_child_weight=5,
        colsample_bytree=1.0,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # 2️⃣ 追加树微调到中国子样本（不是重新fit！）
    mask_china = df_subset['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 追加20棵树
        xgb_reg.fit(X_china, y_china, xgb_model=xgb_reg.get_booster())

    # 3️⃣ 拆分测试集评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out": r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out": explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out": mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }
    return metrics

# === 构造 fel*pe 交互项 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)
df['fel_pe_interaction'] = df['fel'] * df['pe']

feature_subset = ["fel", "pe", "seeo", "fel_pe_interaction"]
target_var = 'peo'
outdir = os.path.join(OUTPUT_DIR, "peo")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(
    evaluate_subset(
        target_var,
        df,
        feature_subset,
        "fel+pe+seeo+fel*pe",
        outdir
    )
)

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(
    os.path.join(outdir, "peo_fel_pe_seeo_felpe回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ fel+pe+seeo+fel*pe变量回归评估完成，结果已保存至：", outdir)

✅ fel+pe+seeo+fel*pe变量回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\peo


### fel+ pe + seeo+fel*seeo

In [19]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(var, df, feature_subset, subset_name, output_dir):
    id_cols = ['Model','Scenario','Region','Year', var]
    df_subset = df[id_cols + feature_subset].dropna()
    X_all, y_all = df_subset[feature_subset], df_subset[var].values

    # 1️⃣ 全部样本训练
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=5,
        min_child_weight=5,
        colsample_bytree=1.0,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # 2️⃣ 追加树微调到中国子样本
    mask_china = df_subset['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 追加20棵树
        xgb_reg.fit(X_china, y_china, xgb_model=xgb_reg.get_booster())

    # 3️⃣ 拆分测试集评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out": r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out": explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out": mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }
    return metrics

# === 构造 fel*seeo 交互项 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)
df['fel_seeo_interaction'] = df['fel'] * df['seeo']

feature_subset = ["fel", "pe", "seeo", "fel_seeo_interaction"]
target_var = 'peo'
outdir = os.path.join(OUTPUT_DIR, "peo")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(
    evaluate_subset(
        target_var,
        df,
        feature_subset,
        "fel+pe+seeo+fel*seeo",
        outdir
    )
)

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(
    os.path.join(outdir, "peo_fel_pe_seeo_felseeo回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ fel+pe+seeo+fel*seeo变量回归评估完成，结果已保存至：", outdir)

✅ fel+pe+seeo+fel*seeo变量回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\peo


### fel+ pe + seeo+fel*pe+fel*seeo

In [20]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(var, df, feature_subset, subset_name, output_dir):
    id_cols = ['Model','Scenario','Region','Year', var]
    df_subset = df[id_cols + feature_subset].dropna()
    X_all, y_all = df_subset[feature_subset], df_subset[var].values

    # 1️⃣ 全部样本训练
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=5,
        min_child_weight=5,
        colsample_bytree=1.0,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # 2️⃣ 追加树微调到中国子样本
    mask_china = df_subset['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 追加20棵树
        xgb_reg.fit(X_china, y_china, xgb_model=xgb_reg.get_booster())

    # 3️⃣ 拆分测试集评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out": r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out": explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out": mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }
    return metrics

# === 构造交互项 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)
df['fel_pe_interaction'] = df['fel'] * df['pe']
df['fel_seeo_interaction'] = df['fel'] * df['seeo']

feature_subset = ["fel", "pe", "seeo", "fel_pe_interaction", "fel_seeo_interaction"]
target_var = 'peo'
outdir = os.path.join(OUTPUT_DIR, "peo")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(
    evaluate_subset(
        target_var,
        df,
        feature_subset,
        "fel+pe+seeo+fel*pe+fel*seeo",
        outdir
    )
)

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(
    os.path.join(outdir, "peo_fel_pe_seeo_felpe_felseeo回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ fel+pe+seeo+fel*pe+fel*seeo变量回归评估完成，结果已保存至：", outdir)

✅ fel+pe+seeo+fel*pe+fel*seeo变量回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\peo


# seeo

In [8]:
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# 单变量调用
analyze('seeo', df, OUTPUT_DIR)

[✔] seeo 使用 XGBoost 分析完成，结果保存在：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo


## 子集特征再训练与验证¶

In [34]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)

def evaluate_subset(var, df, feature_subset, subset_name, output_dir):
    id_cols = ['Model','Scenario','Region','Year', var]
    df = df[id_cols + feature_subset].dropna()
    X_all, y_all = df[feature_subset], df[var].values

    # 1️⃣ 全样本基线模型（40棵树）
    xgb_reg = xgb.XGBRegressor(
        n_estimators=40,
        max_depth=6,
        min_child_weight=5,
        colsample_bytree=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # 2️⃣ 非覆盖式中国子样本微调（增树到60棵树）
    mask_china = df['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg_finetune = xgb.XGBRegressor(
            n_estimators=60,  # 比基线多20棵
            max_depth=6,
            min_child_weight=5,
            colsample_bytree=0.8,
            random_state=42,
            verbosity=0,
            n_jobs=-1,
            tree_method='auto'
        )
        # 关键点：用 xgb_model=基线booster实现“增树”而不是覆盖
        xgb_reg_finetune.fit(X_china, y_china, xgb_model=xgb_reg.get_booster())
        xgb_reg = xgb_reg_finetune

    # 3️⃣ 拆分测试集评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out": r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out": explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out": mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }
    return metrics

In [35]:
linear_feats = ["csc"]
nonlinear_feats = ["peowc", "seebwoc"]
shap_top5_feats = ["peowc", "seebwoc", "csc", "ecese", "pen","pe","see","penr","peowoc","seen"]

In [36]:
# 加载数据
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

target_var = 'seeo'
outdir = os.path.join(OUTPUT_DIR, "seeo")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(evaluate_subset(target_var, df, linear_feats, "线性主效变量", outdir))
results.append(evaluate_subset(target_var, df, nonlinear_feats, "非线性交互变量", outdir))
results.append(evaluate_subset(target_var, df, shap_top5_feats, "SHAP前十变量", outdir))

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(outdir, "seeo_子集建模比较结果.csv"), index=False, encoding='utf-8-sig')
print("✅ 子集建模评估完成，结果已保存至：", outdir)

✅ 子集建模评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo


## Sobol 漏斗式验证

In [12]:
# 选定目标变量和特征组合（例如基于 Morris 筛选）
target_var = "seeo"
sobol_vars = ["peowc", "seebwoc"]  # 可根据 Morris σ 高变量选择

# 数据准备
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# 仅取中国区域数据
fine_tune_regions = ["CHN", "R10CHINA+"]
df_china = df[df['Region'].isin(fine_tune_regions)].dropna(subset=[target_var] + sobol_vars)

X = df_china[sobol_vars]
y = df_china[target_var].values

In [13]:
run_sobol_analysis(X, y, sobol_vars, OUTPUT_DIR, tag="seeo_sobol")

C:\Users\phc\AppData\Local\Temp\ipykernel_20196\633809096.py:17: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 512, calc_second_order=True)


[✔] Sobol 总效应分析完成，结果已保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo_sobol_sobol.csv


C:\Users\phc\AppData\Roaming\Python\Python310\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


## 模型降维&交互建模

### 仅用 peowc

In [21]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(target_var, df, feature_subset, subset_name, output_dir, max_depth=5):
    id_cols = ['Model', 'Scenario', 'Region', 'Year', target_var]
    df_sub = df[id_cols + feature_subset].dropna()
    X_all, y_all = df_sub[feature_subset], df_sub[target_var].values

    # 1️⃣ 基础模型训练
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=max_depth,
        min_child_weight=5,
        colsample_bytree=0.8,
        subsample=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # 2️⃣ 追加树微调到中国子样本
    mask_china = df_sub['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 追加20棵树
        xgb_reg.fit(X_china, y_china, xgb_model=xgb_reg.get_booster())

    # 3️⃣ 拆分测试集做评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out": r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out": explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out": mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }
    return metrics

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 只用 peowc 做回归 ===
feature_subset = ['peowc']
target_var = 'seeo'
outdir = os.path.join(OUTPUT_DIR, "seeo_peowc")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(
    evaluate_subset(
        target_var=target_var,
        df=df,
        feature_subset=feature_subset,
        subset_name="只用 peowc",
        output_dir=outdir,
        max_depth=5
    )
)

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(
    os.path.join(outdir, "seeo_peowc_单变量回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ 只用 peowc 变量回归评估完成，结果已保存至：", outdir)

✅ 只用 peowc 变量回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo_peowc


### 对 peowc 做三段哑变量+peowc:

In [22]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(target_var, df, feature_subset, subset_name, max_depth=5):
    # 只保留 id 列 + 特征 + 目标，去掉缺失
    id_cols = ['Model', 'Scenario', 'Region', 'Year', target_var]
    df_sub = df[id_cols + feature_subset].dropna()
    X_all, y_all = df_sub[feature_subset], df_sub[target_var].values

    # 1️⃣ 全部样本训练
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=max_depth,
        min_child_weight=5,
        colsample_bytree=0.8,
        subsample=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # 2️⃣ 追加树微调中国子样本
    mask_china = df_sub['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_ch, y_ch = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 增加20棵树
        xgb_reg.fit(X_ch, y_ch, xgb_model=xgb_reg.get_booster())

    # 3️⃣ 拆分测试集做评估
    Xtr, Xte, ytr, yte = train_test_split(
        X_all, y_all, test_size=0.3, random_state=42
    )

    return {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in":     r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out":    r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out":   explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out":   mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out":   mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 对 peowc 做三段哑变量 + 原始 peowc ===
df['peowc_low']  = (df.peowc <= 0.25).astype(int)
df['peowc_mid']  = ((df.peowc > 0.25) & (df.peowc <= 1.0)).astype(int)
df['peowc_high'] = (df.peowc > 1.0).astype(int)

feature_subset = ['peowc_low', 'peowc_mid', 'peowc_high', 'peowc']
target_var     = 'seeo'
outdir         = os.path.join(OUTPUT_DIR, "seeo_peowc_dummies")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(
    evaluate_subset(
        target_var=target_var,
        df=df,
        feature_subset=feature_subset,
        subset_name="peowc_三段哑变量+原始",
        max_depth=5
    )
)

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(
    os.path.join(outdir, "seeo_peowc_三段哑变量加原始回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ peowc 三段哑变量+原始变量回归评估完成，结果已保存至：", outdir)

✅ peowc 三段哑变量+原始变量回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo_peowc_dummies


### peowc+seebwoc

In [23]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(target_var, df, feature_subset, subset_name, max_depth=5):
    id_cols = ['Model', 'Scenario', 'Region', 'Year', target_var]
    df_sub = df[id_cols + feature_subset].dropna()
    X_all, y_all = df_sub[feature_subset], df_sub[target_var].values

    # 1️⃣ 全样本训练
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=max_depth,
        min_child_weight=5,
        colsample_bytree=0.8,
        subsample=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # 2️⃣ 中国子集追加树微调
    mask_china = df_sub['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_ch, y_ch = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 追加20棵树
        xgb_reg.fit(X_ch, y_ch, xgb_model=xgb_reg.get_booster())

    # 3️⃣ 评估
    Xtr, Xte, ytr, yte = train_test_split(
        X_all, y_all, test_size=0.3, random_state=42
    )

    return {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in":     r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out":    r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out":   explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out":   mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out":   mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 特征：peowc + seebwoc ===
feature_subset = ['peowc', 'seebwoc']
target_var     = 'seeo'
outdir         = os.path.join(OUTPUT_DIR, "seeo_peowc_seebwoc")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(
    evaluate_subset(
        target_var=target_var,
        df=df,
        feature_subset=feature_subset,
        subset_name="peowc + seebwoc",
        max_depth=5
    )
)

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(
    os.path.join(outdir, "seeo_peowc_seebwoc回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ peowc + seebwoc 回归评估完成，结果已保存至：", outdir)

✅ peowc + seebwoc 回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo_peowc_seebwoc


### peowc+seebwoc+seebwoc*cscf

In [24]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(target_var, df, feature_subset, subset_name, max_depth=5):
    id_cols = ['Model', 'Scenario', 'Region', 'Year', target_var]
    df_sub = df[id_cols + feature_subset].dropna()
    X_all, y_all = df_sub[feature_subset], df_sub[target_var].values

    # 1️⃣ 全样本基础模型
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=max_depth,
        min_child_weight=5,
        colsample_bytree=0.8,
        subsample=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # 2️⃣ 中国子集微调：追加树
    mask_china = df_sub['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_ch, y_ch = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 追加20棵树
        xgb_reg.fit(X_ch, y_ch, xgb_model=xgb_reg.get_booster())

    # 3️⃣ 评估
    Xtr, Xte, ytr, yte = train_test_split(
        X_all, y_all, test_size=0.3, random_state=42
    )

    return {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in":     r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out":    r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out":   explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out":   mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out":   mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# 构造交互特征 seebwoc × cscf
df['seebwoc_x_cscf'] = df['seebwoc'] * df['cscf']

# === 用 peowc、seebwoc、seebwoc_x_cscf 做回归 ===
feature_subset = ['peowc', 'seebwoc', 'seebwoc_x_cscf']
target_var     = 'seeo'
outdir         = os.path.join(OUTPUT_DIR, "seeo_peowc_seebwoc_seebwocxcscf")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(
    evaluate_subset(
        target_var=target_var,
        df=df,
        feature_subset=feature_subset,
        subset_name="peowc + seebwoc + seebwoc*cscf",
        max_depth=5
    )
)

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(
    os.path.join(outdir, "seeo_peowc_seebwoc_seebwocxcscf回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ peowc + seebwoc + seebwoc*cscf 交互回归评估完成，结果已保存至：", outdir)

✅ peowc + seebwoc + seebwoc*cscf 交互回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo_peowc_seebwoc_seebwocxcscf


### peowc+seebwoc+peowc*csc

In [25]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(target_var, df, feature_subset, subset_name, max_depth=5):
    id_cols = ['Model', 'Scenario', 'Region', 'Year', target_var]
    df_sub = df[id_cols + feature_subset].dropna()
    X_all, y_all = df_sub[feature_subset], df_sub[target_var].values

    # 1️⃣ 全样本训练
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=max_depth,
        min_child_weight=5,
        colsample_bytree=0.8,
        subsample=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # 2️⃣ 中国子集“追加树”微调
    mask_china = df_sub['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_ch, y_ch = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20  # 追加20棵树
        xgb_reg.fit(X_ch, y_ch, xgb_model=xgb_reg.get_booster())

    # 训练/测试拆分与评估
    Xtr, Xte, ytr, yte = train_test_split(
        X_all, y_all, test_size=0.3, random_state=42
    )

    return {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in":     r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out":    r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out":   explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out":   mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out":   mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# 构造交互特征 peowc × csc
df['peowc_x_csc'] = df['peowc'] * df['csc']

# === 用 peowc, seebwoc, peowc_x_csc 做回归 ===
feature_subset = ['peowc', 'seebwoc', 'peowc_x_csc']
target_var     = 'seeo'
outdir         = os.path.join(OUTPUT_DIR, "seeo_peowc_seebwoc_peowcxcsc")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(
    evaluate_subset(
        target_var=target_var,
        df=df,
        feature_subset=feature_subset,
        subset_name="peowc + seebwoc + peowc*csc",
        max_depth=5
    )
)

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(
    os.path.join(outdir, "seeo_peowc_seebwoc_peowcxcsc回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ peowc + seebwoc + peowc*csc 回归评估完成，结果已保存至：", outdir)

✅ peowc + seebwoc + peowc*csc 回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo_peowc_seebwoc_peowcxcsc


### peowc+seebwoc+peowc*csc+seebwoc*cscf

In [26]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def evaluate_subset(target_var, df, feature_subset, subset_name, max_depth=5):
    id_cols = ['Model', 'Scenario', 'Region', 'Year', target_var]
    df_sub = df[id_cols + feature_subset].dropna()
    X_all, y_all = df_sub[feature_subset], df_sub[target_var].values

    # 全部样本训练100棵树
    xgb_reg = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=max_depth,
        min_child_weight=5,
        colsample_bytree=0.8,
        subsample=0.8,
        random_state=42,
        verbosity=0,
        n_jobs=-1,
        tree_method='auto'
    )
    xgb_reg.fit(X_all, y_all)

    # 增量微调：中国区域，再加20棵树
    mask_china = df_sub['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_ch, y_ch = X_all[mask_china], y_all[mask_china]
        xgb_reg.n_estimators += 20
        xgb_reg.fit(X_ch, y_ch, xgb_model=xgb_reg.get_booster())

    # 拆分训练/测试并评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    return {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in":     r2_score(ytr, xgb_reg.predict(Xtr)),
        "R2_out":    r2_score(yte, xgb_reg.predict(Xte)),
        "EVS_out":   explained_variance_score(yte, xgb_reg.predict(Xte)),
        "MSE_out":   mean_squared_error(yte, xgb_reg.predict(Xte)),
        "MAE_out":   mean_absolute_error(yte, xgb_reg.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, xgb_reg.predict(Xte)),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# 构造交互特征 peowc×csc 和 seebwoc×cscf
df['peowc_x_csc']    = df['peowc']   * df['csc']
df['seebwoc_x_cscf'] = df['seebwoc'] * df['cscf']

# === 用 peowc, seebwoc, peowc_x_csc, seebwoc_x_cscf 做回归 ===
feature_subset = ['peowc', 'seebwoc', 'peowc_x_csc', 'seebwoc_x_cscf']
target_var     = 'seeo'
outdir         = os.path.join(OUTPUT_DIR, "seeo_full_interactions")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(
    evaluate_subset(
        target_var=target_var,
        df=df,
        feature_subset=feature_subset,
        subset_name="peowc + seebwoc + peowc*csc + seebwoc*cscf",
        max_depth=5
    )
)

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(
    os.path.join(outdir, "seeo_peowc_seebwoc_peowcxcsc_seebwocxcscf_results.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ peowc + seebwoc + peowc*csc + seebwoc*cscf 回归评估完成，结果已保存至：", outdir)

✅ peowc + seebwoc + peowc*csc + seebwoc*cscf 回归评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo_full_interactions
